<span style="color:blue">Thanks for using Drogon for your interactive Spark application. We update Drogon/SparkMagic as often as possible to make it easier, faster and more reliable for you. Have a question or feedback? Ping us on [uChat](https://uchat.uberinternal.com/uber/channels/spark).</span>

What's New
- Now you can use `%%configure` and `%%spark` magics to configure and start a Spark session (deprecating hard-to-use `%load_ext sparkmagic.magics` and `manage_spark` magics). Check out [this example](https://workbench.uberinternal.com/explore/knowledge/localfile/cwang/sparkmagic_python2_example.ipynb) for more details.
- Improved `%%configure` magic. You now can use it to make all Spark and Drogon configurations from within notebook itself. Check out our [latest documentation & examples](https://docs.google.com/document/d/1mkYtDHquh4FjqTeA0Fxii8lyV-P6qzmoABhmmRwm_00/edit#heading=h.xn14pmoorsn0) for more details.
- Bug fixes and performance updates.


In [ ]:
%%configure -f
{
  "kind": "spark", 
  "proxyUser": "dhruven.vora", 
  "sparkEnv": "SPARK_24", 
  "driverMemory": "12g", 
  "queue": "maps_route_analytics", 
  "numExecutors": 100, 
  "executorCores": 1, 
  "driverCores": 4,
  "conf": {
    "spark.driver.maxResultSize": "10g",
    "spark.executor.memoryOverhead": 3072, 
    "spark.locality.wait": "0",
    "spark.default.parallelism":10000
  },
  "executorMemory": "24g",
  "drogonHeaders": {
    "X-DROGON-CLUSTER": "PHX2/Secure"
  }
}

In [ ]:
%%spark

In [ ]:
/**
 * This section defines all the objects will be used in the following algorithm.
 */
case class Location (
    latitude: Double,
    longitude: Double
) extends Serializable

case class Point (
    latE7: Integer,
    lngE7: Integer
) extends Serializable

case class Polyline (
    points: List[Location]
) extends Serializable

case class Geometry(
    polyline: Polyline
) extends Serializable

case class DirectedSegment (
    uuid: String,
    direction: String
) extends Serializable

case class Segment (
    startJunctionUuid: String,
    endJunctionUuid: String 
) extends Serializable

case class TurnRestrictionMapFeature (
    uuid: String,
    segments: List[DirectedSegment]    
) extends Serializable

case class TurnRestrictionMapFeatures (
    features: List[TurnRestrictionMapFeature]    
) extends Serializable

case class SegmentMapFeature(
    uuid: String,
    locations: List[Location]
) extends Serializable

case class JoinedManeuvers (
    uuid: String,
    segments: List[DirectedSegment],
    segment_lines: List[SegmentMapFeature]
) extends Serializable


In [ ]:
/**
 * Input params for the algorithm
 */
val startDate= "2022-12-11"
val endDate = "2022-12-17"
val ummVersion = "f905b99a-8137-11ed-9118-000af7d19b40"

In [ ]:
/**
this class loads maneuvers map features from umm.map_feature_maneuvers_tomtom
*/

import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import spark.implicits._
import org.apache.spark.sql.functions._
import scala.collection.mutable.ListBuffer
import org.apache.spark.sql._

object TurnRestrictionMapFeatureLoader extends Serializable {

  /** Run query to load trips from the table by city and day
    * @param utcDateStr
    * @param cityIds
    * */
  def load(builduuid: String): DataFrame = {

    var query =
      s"""select 
         | uuid,
         | data.maneuver.segments as segments
         | from umm.map_feature_maneuvers_tomtom
         | where builduuid = '$builduuid'
         | and data.maneuver.type in ('FORBIDDEN_MANEUVER','FORBIDDEN_U_TURN')""".stripMargin
        .replaceAll("\n", " ")

    spark.sql(query)
  }

  /** Store dataset in the right schema
    * @param rawDataset
    * */
  def makeDataset(rawDataset: DataFrame): Dataset[TurnRestrictionMapFeature] = {

    rawDataset.map(r => {
        var segments = ListBuffer[DirectedSegment]()
        r.getAs[Seq[Row]]("segments").foreach(row => segments += DirectedSegment(
                                                                    row.getAs[String]("uuid"),
                                                                    row.getAs[String]("direction")
                                                                )
                                             )
        
        val subSegments = segments.toList.sliding(2).toList
        
        var result = ListBuffer[TurnRestrictionMapFeature]()
        
        subSegments.foreach(ss => result += TurnRestrictionMapFeature(
            uuid = r.getAs[String]("uuid"),
            segments = ss
          ))
        
        TurnRestrictionMapFeatures(result.toList)
    }).flatMap(_.features)
  }
}

In [ ]:
/**
load maneuvers map features from umm.map_feature_maneuvers_tomtom
*/
val turnRestrictionMapFeaturesRaw = TurnRestrictionMapFeatureLoader.load(ummVersion)
val turnRestrictionMapFeatures = TurnRestrictionMapFeatureLoader.makeDataset(turnRestrictionMapFeaturesRaw).cache()
turnRestrictionMapFeatures.count()

In [ ]:
/**
this class loads segments map features from umm.map_feature_segments_tomtom
*/

import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import spark.implicits._
import org.apache.spark.sql.functions._
import scala.collection.mutable.ListBuffer
import org.apache.spark.sql._


object SegmentMapFeatureLoader extends Serializable {

  /** Run query to load trips from the table by city and day
    * @param utcDateStr
    * @param cityIds
    * */
  def load(builduuid: String): DataFrame = {

    var query =
      s"""select 
         | uuid,
         | data.segment as segment,
         | geometry
         | from umm.map_feature_segments_tomtom
         | where builduuid = '$builduuid'""".stripMargin
        .replaceAll("\n", " ")

    spark.sql(query)
  }

  /** Store dataset in the right schema
    * @param rawDataset
    * */
  def makeDataset(rawDataset: DataFrame): Dataset[SegmentMapFeature] = {

    rawDataset.map(r => {
        
        val segment = Segment(r.getAs[Row]("segment").getAs[String]("startJunctionUuid"),
                              r.getAs[Row]("segment").getAs[String]("endJunctionUuid")
                      )
        
        var locations = ListBuffer[Location]()
        val polylineRaw = r.getAs[Row]("geometry").getAs[Row]("polyline")
        if(polylineRaw != null) {
            polylineRaw.getAs[Seq[Row]]("points").foreach(pt => {
                    val lat = pt.getAs[Integer]("latE7") / 1.0E7
                    val long = pt.getAs[Integer]("lngE7") / 1.0E7
                    locations += Location(lat, long)
                }
            )
        }
        
        SegmentMapFeature(
            r.getAs[String]("uuid"),
            locations.toList
          )
    })
  }
}

In [ ]:
/**
load segments map features from umm.map_feature_segments_tomtom
*/
val segmentMapFeaturesRaw = SegmentMapFeatureLoader.load(ummVersion)
val segmentMapFeatures = SegmentMapFeatureLoader.makeDataset(segmentMapFeaturesRaw).cache()
segmentMapFeatures.count()

In [ ]:
// Join maneuvers and segments to associate polylines to maneuvers.
val joinedManeuvers = turnRestrictionMapFeatures.withColumn("segment", explode(col("segments"))).alias("M").
                        joinWith(segmentMapFeatures.alias("S"), col("M.segment.uuid")===col("S.uuid")).
                        groupBy(col("_1.uuid"),col("_1.segments")).
                        agg(collect_list("_2").as("segment_lines")).
                        cache()

In [ ]:
// Utils class to calculate turn angle between two segments.
object Utils {
    
    def convertDeltaLongitudeToMeters(atLatitude: Double): Double = {
        Math.cos(Math.toRadians(atLatitude)) * 111319.49079327357
    }
    
    def approxAtan2(y: Double, x: Double): Double = {
        var absY:Double = Math.abs(y);
        var r:Double = 1.0;
        var angle:Double = 0.0;
        var xPlusAbsY:Double  = x + absY;
        var xMinusAbsY:Double  = x - absY;
        if (x < 0.0) {
          if (xMinusAbsY == 0.0) {
            // avoid divide by zero
            xMinusAbsY = 1E-12;
          }
          r = -xPlusAbsY / xMinusAbsY;
          angle = 2.356194490192345;
        } else {
          if (xPlusAbsY == 0.0) {
            // avoid divide by zero
            xPlusAbsY = 1E-12;
          }
          r = xMinusAbsY / xPlusAbsY;
          angle = 0.7853981633974483;
        }
        angle += (0.1963 * r * r - 0.9817) * r;
        // negate if in quadrant III or IV
        if(y < 0.0) {
            return  -angle;
        } else {
            return angle
        }
    }
    
    def fastApproxAzimuth(point1: Location, point2: Location): Option[Double] = {
        
        val verticalDistance:Double = (point2.latitude - point1.latitude) * 111319.49079327357;
        val horizontalDistance:Double = (point2.longitude - point1.longitude) * convertDeltaLongitudeToMeters(point1.latitude);

        var angleDegrees:Double =
            Math.toDegrees(approxAtan2(verticalDistance, horizontalDistance));

        // atan2 is on a standard unit circle - that means an angle of 0 corresponds to East and
        // increases as you go counter-clockwise.  According to the official Azimuth definition, we
        // want East to be 90, and increase as you go clockwise.
        angleDegrees = 90 - angleDegrees;
        return wrapAngle180(angleDegrees);
    }
    
    def wrapAngle180(angleDegreesParam: Double): Option[Double] = {
        val angleDegrees = angleDegreesParam % 360;
        if(angleDegrees < -360 && angleDegrees > 360) {
            return Option.empty
        }
        // wrapping to [-180,180)
        if (angleDegrees >= 180) {
          return Some(angleDegrees - 360);
        } else if (angleDegrees < -180) {
          return Some(angleDegrees + 360);
        } else {
          return Some(angleDegrees);
        }
    }
}

In [ ]:
// compute the angle between two segments
val angles = joinedManeuvers.
filter(man => man.getAs[String]("uuid") == "63c38201-7161-949d-4764-200176e7687a").
map(man => {
    val segmentLines = man.getAs[Seq[Row]]("segment_lines")
    
    var segments = ListBuffer[SegmentMapFeature]()
    segmentLines.foreach(segment => {
        var locations = ListBuffer[Location]()
        val polyline = segment.getAs[Seq[Row]]("locations")
        polyline.foreach(pt => {
                val lat = pt.getAs[Double]("latitude")
                val long = pt.getAs[Double]("longitude")
                locations += Location(lat, long)
            }
        )
        
        segments += SegmentMapFeature("", locations.toList)
    })
    
    val point1 = segments.head.locations.head
    val point2 = segments.head.locations.last
    val point3 = segments.last.locations.head
    val point4 = segments.last.locations.last
    
    val bearing1 = Utils.fastApproxAzimuth(point1, point2)
    val bearing2 = Utils.fastApproxAzimuth(point3, point4)
        
    if(bearing1.isDefined && bearing2.isDefined) {
        Utils.wrapAngle180(bearing2.get - bearing1.get)
    }
    else {
        None
    }
}).
filter(r => r.isDefined).
map(angle => Math.round(angle.get)).
cache()


In [ ]:
val total = angles.count()

In [ ]:
angles.collect().foreach(println)